In [4]:
import pandas as pd
import metapy
import boto3

In [5]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='julibdnews', Key=u'newnews.csv')
df = pd.read_csv(obj['Body'])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
inv_idx = metapy.index.make_inverted_index('news.toml')

In [ ]:
inv_idx.num_docs()

In [ ]:
inv_idx.unique_terms()

In [ ]:
inv_idx.avg_doc_length()

In [ ]:
inv_idx.index_name()

In [ ]:
inv_idx.total_corpus_terms()

In [ ]:
ranker = metapy.index.OkapiBM25()

In [ ]:
query = metapy.index.Document()
query.content("donald trump and gardens")

In [ ]:
top_docs = ranker.score(inv_idx, query, num_results=5)
top_docs

In [ ]:
for num, (d_id, _) in enumerate(top_docs):
    content = inv_idx.metadata(d_id).get('content')
    print("{}. {}...\n".format(num + 1, content[0:250]))

# Topicos

In [ ]:
fidx = metapy.index.make_forward_index('news.toml')

In [ ]:
dset = metapy.classify.MulticlassDataset(fidx)

In [ ]:
model = metapy.topics.LDAParallelGibbs(docs=dset, num_topics=10, alpha=0.1, beta=0.1)
model.run(num_iters=100)
model.save('lda-news')

In [ ]:
model = metapy.topics.TopicModel('lda-news')

In [ ]:
for topic in range(0, model.num_topics()):
    print("Topic {}:".format(topic + 1))
    for tid, val in model.top_k(topic, 10, metapy.topics.BLTermScorer(model)):
        print("{}: {}".format(fidx.term_text(tid), val))
    print("======\n")

In [ ]:
import pandas as pd

data = []
for doc in dset:
    proportions = model.topic_distribution(doc.id)
    data.append([dset.label(doc)] + [proportions.probability(i) for i in range(0, model.num_topics())])
df_topicos = pd.DataFrame(data, columns=['label'] + ["Topic {}".format(i + 1) for i in range(0, model.num_topics())])

In [ ]:
df_topicos.head()

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

for i in range(0, model.num_topics()):
    print("Topic {}".format(i + 1))
    sns.swarmplot(data=df_topicos, x='label', y="Topic {}".format(i + 1))
    plt.show()

In [ ]:
file1 = open("/home/hadoop/st1800eafit-final/data/news/news.dat","w")
for index, row in df.iterrows():
    #print(row['title'], row['content'], row['author'])
    texto = str(row['title']) + ' ' + str(row['content']) + ' ' + str(row['author'])
    texto = texto.replace("\n", " ")
    file1.writelines(texto+'\n')
file1.close()

In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("yarn-client")
sc = SparkContext()
print(sc.version)

2.4.3


In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
data = sqlContext.read.format("csv")

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('app-sentimientos').getOrCreate()

In [6]:
df.to_csv("/home/hadoop/news.csv")

In [31]:
os.getcwd()

'/home/hadoop/st1800eafit-final'

In [36]:
# importing some libraries
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
# stuff we'll need for text processing
from nltk.corpus import stopwords
import re as re
from pyspark.ml.feature import CountVectorizer , IDF
# stuff we'll need for building the model

from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
# reading the data
data = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load(os.path.realpath("/home/hadoop/news.csv"))

In [40]:
reviews = data.rdd.map(lambda x : x['content']).filter(lambda x: x is not None)

In [42]:
reviews = data.rdd.map(lambda x : x['content']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
tokens = reviews                                                   \
    .map( lambda document: document.strip().lower())               \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \
    .zipWithIndex()

In [44]:
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])
# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=10.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 

In [54]:


lda_model = LDA.train(result_tfidf[['index','features']], k=num_topics, maxIterations=max_iterations)

Py4JError: An error occurred while calling o535.trainLDAModel. Trace:
py4j.Py4JException: Method trainLDAModel([class org.apache.spark.sql.Dataset, class java.lang.Integer, class java.lang.Integer, class java.lang.Double, class java.lang.Double, null, class java.lang.Integer, class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [51]:
num_topics = 10
max_iterations = 100
r
lda_model = LDA.train(result_tfidf[['index','features']].map(list), k=num_topics, maxIterations=max_iterations)

AttributeError: 'DataFrame' object has no attribute 'map'